# DATA

In [1]:
from data.preprocessing import *

In [2]:
scrap_date = ['20230921', '20230922','20230923', '20230924']

In [3]:
import_train(scrap_date)

100%|██████████| 377922/377922 [00:00<00:00, 483025.67it/s]


100%|██████████| 183610480/183610480 [03:16<00:00, 935089.18it/s] 


100%|██████████| 422672/422672 [00:01<00:00, 417331.95it/s]


100%|██████████| 183590176/183590176 [03:03<00:00, 999370.92it/s] 


In [3]:
independent = level_1_preprocessing()
independent

array([[-446.075     ,   47.2625    ,    8.3375    , ...,    0.73001426,
          17.871565  ,    9.731378  ],
       [-446.1       ,   46.433334  ,    8.433333  , ...,    0.6353522 ,
          15.668235  ,    6.660771  ],
       [-446.11252   ,   46.675003  ,    8.675     , ...,    0.5244114 ,
          12.89051   ,    7.266075  ],
       ...,
       [-449.5       ,    9.971429  ,  -15.785714  , ...,    8.531774  ,
          46.694794  ,  271.5274    ],
       [-450.91428   ,    9.928572  ,  -18.942858  , ...,    6.4501734 ,
          39.44411   ,  267.90884   ],
       [-453.02222   ,   11.711111  ,  -18.533333  , ...,    6.371899  ,
          38.000023  ,  267.42593   ]], dtype=float32)

In [4]:
day = [int(i[-2:]) for i in scrap_date]
month = [scrap_date[0][:6]]

In [ ]:
import_Dst(month)

In [5]:
import_Dst(month)

dependent = day_Dst(months = month, days = day)

dependent

0     -9.0
1     -6.0
2     -6.0
3     -2.0
4     -4.0
      ... 
91    -7.0
92     1.0
93     9.0
94   -21.0
95   -42.0
Name: Dst, Length: 96, dtype: object

## 1 hour based data example

In [6]:
hour_independent = independent.resample('60min').mean()

In [7]:
dependent.index = hour_independent.index
hour_dataset = pd.concat([hour_independent, dependent], axis = 1)

In [10]:
hour_dataset

,proton_vx_gse,proton_vy_gse,proton_vz_gse,proton_vx_gsm,proton_vy_gsm,proton_vz_gsm,proton_speed,proton_density,proton_temperature,bt,...,by_gse,bz_gse,theta_gse,phi_gse,bx_gsm,by_gsm,bz_gsm,theta_gsm,phi_gsm,Dst
time,,,,,,,,,,,,,,,,,,,,,
2023-09-21 00:00:00,-440.625397,48.084110,8.653492,-440.625397,48.672775,-4.188692,443.348328,11.342950,16096.352539,2.130791,...,0.847840,1.321617,40.749371,51.207500,1.136005,1.163511,1.053266,30.737850,50.519398,-9.0
2023-09-21 01:00:00,-436.380035,46.447960,7.245828,-436.380035,46.597908,-6.154580,438.915222,11.186322,17029.425781,1.659912,...,1.339047,0.834525,29.932814,104.178558,-0.331430,1.518060,0.422348,14.121505,102.622238,-6.0
2023-09-21 02:00:00,-435.234314,40.302071,6.463599,-435.234314,40.305603,-6.439172,437.146454,12.224415,18000.255859,1.124828,...,0.454947,0.379375,17.462206,148.397659,0.243548,0.554094,0.221591,10.128606,144.417480,-6.0
2023-09-21 03:00:00,-432.174438,42.473843,10.246343,-432.174438,43.409988,-5.206321,434.405090,10.848347,24184.771484,1.241080,...,-0.424566,0.367620,18.017147,269.800659,0.965414,-0.272296,0.490578,23.740959,233.121796,-2.0
2023-09-21 04:00:00,-428.908325,42.942600,16.721794,-428.908325,46.078426,-1.175624,431.389709,9.426498,20835.503906,1.003060,...,0.206462,0.452351,29.159575,196.777145,-0.126963,0.363385,0.332044,25.686102,176.670593,-4.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2023-09-24 19:00:00,-330.359406,15.307604,12.737278,-330.359406,18.400660,7.654412,331.829895,13.369762,47151.992188,8.749233,...,1.051775,-4.039783,-27.158567,163.281403,-4.755493,-0.196672,-4.210385,-30.925808,171.490646,-7.0
2023-09-24 20:00:00,-454.541382,7.437764,-7.265269,-454.541382,5.248778,-9.000386,455.119415,44.795341,156803.968750,23.165615,...,-8.180365,-9.826681,-29.095251,238.537521,3.625593,-10.565377,-7.297751,-22.919752,242.067307,1.0
2023-09-24 21:00:00,-458.739929,28.060617,-16.425770,-458.739929,22.966177,-23.014624,460.051422,42.428162,171521.937500,17.334135,...,1.086052,-12.582829,-54.542412,154.113449,0.717099,-2.141264,-12.428204,-55.938339,203.909363,9.0


## Datasets

There is a class on `data/data_utils` that alligns a given sequence to the next hours for training.
The same for hours.